# Optimization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2020/blob/master/04-hyperparams/optimization.ipynb)


**Цели тетрадки**

1. Познакомиться с процедурой подбора гиперпараметров
3. Протренировать сверточную сеть

**План**

1. Написать функцию для Learning Rate Range Test и отладить ее на FashionMNIST
2. Подобрать параметры и натренировать сверточную сеть на Imagenette


**Настоятельно рекомендуется воспользоваться колабом**

In [ ]:
# install requirements
! pip install torchviz torchvision

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from torchvision.datasets import FashionMNIST
from torchvision import transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])
# имеет смысл добавить нормирование картинок

train_dataset = FashionMNIST("./tmp", train=True, download=True, transform=transform)
val_dataset = FashionMNIST("./tmp", train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=32)

**(0.2 балла)** Допишите тренировочный цикл и проверьте, что он работает на какой-нибудь сверточной сети

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def train(model, optimizer, dataloader): 
    model.to(device)
    model.train()
    logs = defaultdict(list)
    for x, y in tqdm(dataloader):
        <your code here>
        
        # у вас должно быть две скалярных переменных: с метрикой и лоссом
        logs['acc'].append(acc.item())
        logs['loss'].append(loss.item())
    return logs

def validate(model, dataloader):
    model.to(device)
    model.eval()
    logs = defaultdict(list)
    for x, y in tqdm(dataloader):
        <your code>
        # у вас должно быть две скалярных переменных: с метрикой и лоссом
        logs['acc'].append(acc.item())
        logs['loss'].append(loss.item())
    
    return {k: [np.mean(v)] for k, v in logs.items()}

def plot_logs(logs):
    clear_output()
    plt.figure()
    plt.plot(logs['acc'], zorder=1)
    plt.scatter(logs['steps'], logs['val_acc'], marker='+', s=180, c='orange', label='val', zorder=2)
    plt.show()

    plt.figure()
    # для отображения подписей воспользуйтесь label&legend
    # plt.plot(..., label=name)
    # plt.legend() 
    <your code>        
    plt.legend()
    plt.grid()
    plt.show()


def train_model(model, optimizer, train_loader, val_loader, epochs=10):
    logs = defaultdict(list)
    for epoch in range(epochs):
        train_logs = train(model, opt, train_loader)
        
        # вы вольны переписать объединение логов
        for k, v in train_logs.items():
            logs[k].extend(v)

        val_logs = validate(model, val_loader)
        for k, v in val_logs.items():
            logs[f'val_{k}'].extend(v)
        logs['steps'].append(len(logs['loss']))

        clear_output()
        plot_logs(logs)

In [ ]:
cnn = <задайте какую-нибудь сверточную сеть>

opt = torch.optim.SGD(cnn.parameters(), lr=0.01)
train_model(cnn, opt, train_loader, val_loader)

## LRRT (Find LR)

**(0.1 балла)** Напишите функцию Learning Rate Range Test:
    
$$t = \frac{\mathrm{step}}{\mathrm{total}}\\
\mathrm {lr} = \exp((1-t)\log a + t \log b) 
$$

Чтобы поменять LR можно обойти оптимизируемые параметры следующим способом:
```python
for param_group in optimizer.param_groups:
    param_group['lr'] = lr
```

**(0.2 балла)** Постройте графики Loss/Acc vs lr.

Напишите словами, LR из какого диапазона вы выберете для тренировки сети.
И протренируйте вашу сеть с новыми гиперпараметрами.

**NB: вы можете попробовать поменять оптимизатор и параметры в оптимизаторе**

In [ ]:
def find_lr(model, optimizer, dataloader, min_lr, max_lr):
    model.to(device)
    model.train()
    
    logs = defaultdict(list)
    for x, y in tqdm(dataloader):
        <your code>
        
        logs['loss'].append(acc.item())
        logs['acc'].append(loss.item())
        logs['lr'].append(lr)
        
    
    plt.figure()
    plt.plot(logs['lr'], logs['loss'])
    plt.set_xscale('log')
    plt.grid()
    plt.show()
    


find_lr(model, optimizer, train_loader, 1e-6, 10.0)

## Imagenette

Скачаем датасет и напишем загрузчики данных

In [ ]:
! wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
! tar xf imagenette2-160.tgz

In [ ]:
from torchvision.datasets import ImageFolder

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(160),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = ImageFolder("./imagenette2-160/train/", transform=train_transform)
val_dataset = ImageFolder("./imagenette2-160/val/", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

**(0.1 балла)** Преобразуйте выходной слой для классификации на 10 классов

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=False)
# преобразуйте выходной слой
<your code>

**(0.2 балла)** Выберете оптимизатор и определите интересный для тренировки интервал LR 

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=0.01)
find_lr(model, opt, train_loader, 1e-6, 10.0)

**(0.2 балла)** Натренируйте модель с выбранными гиперпараметрами

In [ ]:
model = ...
opt = torch.optim.SGD(model.parameters(), lr=...)
train_model(model, opt, train_loader, val_loader)